# **Jupyter Notebook: Estimating *E. coli* Volume and Molecular Counts**

## **Objective**
In this exercise, we will:
1. Use Napari to visualize microscope images of *E. coli* and a graticule to calculate the scale.
2. Measure the dimensions of an *E. coli* cell and estimate its volume.
3. Calculate the number of molecules present in the cell at a given concentration (e.g., 1 nM).

---

## **Setup**
### **Install Required Libraries**
Before running this notebook, ensure the following libraries are installed:
```bash
pip install napari jupyterlab matplotlib numpy scipy scikit-image
```

### **Files Needed**
- `Ecoli100x.tif`: Microscope image of *E. coli* cells.
- `Graticule100x.tif`: Graticule image used to calibrate the scale.

In [ ]:
# QED to display the images inline to verify that the graticule and E. coli images are the proper scale.
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Load images
img1 = tiff.imread('data/Graticule100x.tif')  # Graticule
img2 = tiff.imread('data/Ecoli100x.tif') # E coli

# Identify the larger and smaller image
if img1.shape > img2.shape:
    large_img, small_img = img1, img2
else:
    large_img, small_img = img2, img1

# Compute cropping coordinates
large_h, large_w = large_img.shape[:2]
small_h, small_w = small_img.shape[:2]

crop_h_start = (large_h - small_h) // 2
crop_w_start = (large_w - small_w) // 2

cropped_large_img = large_img[crop_h_start:crop_h_start + small_h, crop_w_start:crop_w_start + small_w]

# Blend images with transparency
alpha = 0.5  # Adjust opacity
overlay = (1 - alpha) * cropped_large_img + alpha * small_img

# Display the result
plt.figure(figsize=(6,6))
plt.imshow(overlay, cmap="gray")
plt.axis("off")
plt.show()

## **Step 1: Load Images in Napari**
We'll use Napari to load and visualize the microscope images.

In [ ]:
import napari
from skimage import io

# Load the images
ecoli_image = io.imread("Ecoli100x.tif")  # Image of E. coli
graticule_image = io.imread("Graticule100x.tif")  # Graticule for scale calibration

# Open Napari viewer
viewer = napari.Viewer()
viewer.add_image(ecoli_image, name="E. coli")
viewer.add_image(graticule_image, name="Graticule")

### **Instructions**
1. Run the above cell to open Napari within the Jupyter interface.
2. Use the **line tool** in Napari to measure distances on the graticule and *E. coli* images.
3. Record the pixel measurements and convert them to micrometers ($ \mu m $) using the graticule scale.

## **Step 2: Calculate the Scale**
The graticule image contains markings with known dimensions. Use it to determine the scale (pixels per micrometer).

### **Example**
Suppose the graticule markings indicate that 10 pixels correspond to 1 µm. Then:
$$
\text{Scale} = \frac{\text{Number of pixels}}{\text{Known distance in µm}}
$$

In [ ]:
# Example: If 10 pixels = 1 µm
pixels_per_um = 10  # Replace with your measurement from Napari
scale_um_per_pixel = 1 / pixels_per_um
print(f"Scale: {scale_um_per_pixel:.2f} µm/pixel")

## **Step 3: Measure Dimensions of *E. coli***
Use Napari to measure the length ($L$) and diameter ($D$) of an *E. coli* cell in pixels. Convert these measurements to micrometers using the scale.

### **Example**
If the measured length is 20 pixels and the diameter is 5 pixels:
$$
L = 20 \, \text{pixels} \times \text{Scale} \, (\mu m/\text{pixel})
$$
$$
D = 5 \, \text{pixels} \times \text{Scale} \, (\mu m/\text{pixel})
$$

In [ ]:
# Example measurements (replace with your values)
length_pixels = 20  # Length in pixels
diameter_pixels = 5  # Diameter in pixels

# Convert to micrometers
length_um = length_pixels * scale_um_per_pixel
diameter_um = diameter_pixels * scale_um_per_pixel

print(f"Length: {length_um:.2f} µm")
print(f"Diameter: {diameter_um:.2f} µm")

## **Step 4: Calculate the Volume**
Model the *E. coli* cell as a cylinder and calculate its volume:
$$
V = \pi r^2 L
$$
Where:
- $r = D/2$ is the radius,
- $L$ is the length.

In [ ]:
import numpy as np

# Radius
radius_um = diameter_um / 2

# Volume of a cylinder
volume_um3 = np.pi * (radius_um**2) * length_um
print(f"Volume of the E. coli cell: {volume_um3:.2f} µm³")

## **Step 5: Convert Volume to Liters**
Convert the volume from cubic micrometers ($ \mu m^3 $) to liters ($ L $):
$$
1 \, \mu m^3 = 10^{-18} \, \text{m}^3, \quad 1 \, \text{L} = 10^{-3} \, \text{m}^3
$$

In [ ]:
# Conversion factors
volume_m3 = volume_um3 * 1e-18  # Convert µm³ to m³
volume_L = volume_m3 * 1e3      # Convert m³ to L

print(f"Volume of the E. coli cell: {volume_L:.2e} L")

## **Step 6: Calculate the Number of Molecules**
Calculate the number of molecules at a given concentration (e.g., 1 nM):
$$
N = C \cdot V \cdot N_A
$$
Where:
- $C$: Concentration in mol/L,
- $V$: Volume in liters,
- $N_A$: Avogadro's number ($6.022 \times 10^{23}$).

In [ ]:
# Constants
concentration_nM = 1  # Concentration in nanomolar (nM)
avogadro_number = 6.022e23  # Avogadro's number

# Convert concentration to mol/L
concentration_mol_per_L = concentration_nM * 1e-9

# Number of molecules
number_of_molecules = concentration_mol_per_L * volume_L * avogadro_number
print(f"Number of molecules at {concentration_nM} nM: {number_of_molecules:.2f}")

## **Discussion**
### **Key Results**
1. At $ 1 \, \text{nM} $, there are approximately **0.24 molecules** in an *E. coli* cell, highlighting the stochastic nature of low-concentration processes.
2. At $ 1 \, \mu M $, the number of molecules increases to ~241, making molecular interactions more deterministic.

### **Questions for Discussion**
1. How does the calculated volume compare to typical *E. coli* dimensions?
2. What are the implications of having fewer than one molecule in the cell at low concentrations?
3. How might diffusion timescales vary based on the estimated volume?

---

## **Next Steps**
Now that we’ve estimated the number of molecules in a cell, let’s explore how these molecules move and interact through diffusion.